In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Path to the training and testing image folders
train_folder = "/kaggle/input/project/train/train"
test_folder = "/kaggle/input/project/test/test"

# Path to the training and testing labels CSV files
train_labels_file = "/kaggle/input/project/csvTrainLabel 13440x1.csv"
test_labels_file = "/kaggle/input/project/csvTestLabel 3360x1.csv"

# Load and preprocess training data
def load_and_preprocess_data(folder_path, labels_file):
    labels = pd.read_csv(labels_file, header=None, names=['label']).to_numpy()

    images = []
    for i in range(1, len(labels) + 1):
        img_path = os.path.join(folder_path, f"id_{i}_label_{labels[i-1, 0]}.png")
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(32, 32), color_mode='grayscale')
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        images.append(img_array)

    images = np.array(images)
    images = images / 255.0

    labels_one_hot = to_categorical(labels - 1, num_classes=28)

    return images, labels_one_hot

train_images, train_labels = load_and_preprocess_data(train_folder, train_labels_file)
test_images, test_labels = load_and_preprocess_data(test_folder, test_labels_file)
print("Finish")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

# Function to build the CNN model
def build_cnn_model(input_shape=(32, 32, 1), num_classes=28):
    model = models.Sequential()

    # First Convolutional Layer
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, padding='same', strides=(1, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    # Second Convolutional Layer
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same', strides=(1, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    # Third Convolutional Layer
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same', strides=(1, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    # Flatten the output
    model.add(layers.Flatten())

    # Fully Connected Layers
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.6))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.6))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.6))

    # Output Layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Define Hyperparameters
learning_rate = 0.0001
batch_size = 16
epochs = 20

# Create an instance of the Adam optimizer with the specified learning rate
custom_optimizer = Adam(learning_rate=learning_rate)

# Build the CNN model
cnn_model = build_cnn_model()

# Compile the model
cnn_model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model and get the history object
history = cnn_model.fit(
    train_images, 
    train_labels, 
    epochs=epochs, 
    batch_size=batch_size, 
    validation_data=(test_images, test_labels),  # Validation data for each epoch
)

# Evaluate the model on the test set
test_loss, test_accuracy = cnn_model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Plot the training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Testing Accuracy')  # Plot validation accuracy against epochs
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Testing Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Data augmentation --- part 1
# This cell is to fine tune the zoom range on the testing accuercy and loss

# Define a range of zoom levels to test
zoom_ranges = [0.1, 0.2, 0.3, 0.4, 0.5]

# Lists to store testing accuracies and losses for each zoom range
testing_accuracies = []
testing_losses = []

for zoom_range in zoom_ranges:
    # Create an instance of ImageDataGenerator with the current zoom range
    datagen = ImageDataGenerator(
        zoom_range=zoom_range,
        fill_mode='nearest'
    )

    # Apply data augmentation to the entire training dataset
    augmented_train_data = datagen.flow(train_images, train_labels, batch_size=32)

    # Combine original training data with augmented data
    combined_train_data = np.concatenate([train_images, augmented_train_data[0][0]], axis=0)
    combined_train_labels = np.concatenate([train_labels, augmented_train_data[0][1]], axis=0)

    # Create an instance of the Adam optimizer with the specified learning rate
    custom_optimizer = Adam(learning_rate=learning_rate)

    # Build the CNN model
    new_model = build_cnn_model()

    # Compile the model
    new_model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model using combined data
    new_model.fit(
        combined_train_data,
        combined_train_labels,
        batch_size=16,
        epochs=epochs,
        validation_data=(test_images, test_labels),
        verbose=0
    )

    # Evaluate the model on the test dataset
    test_loss, test_accuracy = new_model.evaluate(test_images, test_labels)
    testing_accuracies.append(test_accuracy)
    testing_losses.append(test_loss)

# Plot the testing accuracies against zoom ranges
plt.figure(figsize=(12, 4))

# Plot Testing Accuracy
plt.subplot(1, 2, 1)
plt.plot(zoom_ranges, testing_accuracies, marker='o')
plt.xlabel('Zoom Range')
plt.ylabel('Testing Accuracy')
plt.title('Effect of Zoom Range on Testing Accuracy')

# Plot Testing Loss
plt.subplot(1, 2, 2)
plt.plot(zoom_ranges, testing_losses, marker='o', color='orange')
plt.xlabel('Zoom Range')
plt.ylabel('Testing Loss')
plt.title('Effect of Zoom Range on Testing Loss')

plt.tight_layout()
plt.show()


In [ ]:
# Data augmentation --- part 2
# This cell is to fine tune the width shift range on the testing accuercy and loss

# Define a range of zoom levels to test
width_shift_ranges = [0.1, 0.2, 0.3, 0.4, 0.5]

# Lists to store testing accuracies and losses for each zoom range
testing_accuracies = []
testing_losses = []

for width_range in width_shift_ranges:
    # Create an instance of ImageDataGenerator with the current zoom range
    datagen = ImageDataGenerator(
        zoom_range=0.5,
        fill_mode='nearest',
        width_shift_range=width_range
    )

    # Apply data augmentation to the entire training dataset
    augmented_train_data = datagen.flow(train_images, train_labels, batch_size=32)

    # Combine original training data with augmented data
    combined_train_data = np.concatenate([train_images, augmented_train_data[0][0]], axis=0)
    combined_train_labels = np.concatenate([train_labels, augmented_train_data[0][1]], axis=0)

    # Create an instance of the Adam optimizer with the specified learning rate
    custom_optimizer = Adam(learning_rate=learning_rate)

    # Build the CNN model
    new_model = build_cnn_model()

    # Compile the model
    new_model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model using combined data
    new_model.fit(
        combined_train_data,
        combined_train_labels,
        batch_size=16,
        epochs=epochs,
        validation_data=(test_images, test_labels),
        verbose=0
    )

    # Evaluate the model on the test dataset
    test_loss, test_accuracy = new_model.evaluate(test_images, test_labels)
    testing_accuracies.append(test_accuracy)
    testing_losses.append(test_loss)

# Plot the testing accuracies against zoom ranges
plt.figure(figsize=(12, 4))

# Plot Testing Accuracy
plt.subplot(1, 2, 1)
plt.plot(width_shift_ranges, testing_accuracies, marker='o')
plt.xlabel('Width Shift Range')
plt.ylabel('Testing Accuracy')
plt.title('Effect of Width Shift Range on Testing Accuracy')

# Plot Testing Loss
plt.subplot(1, 2, 2)
plt.plot(width_shift_ranges, testing_losses, marker='o', color='orange')
plt.xlabel('Width Shift Range')
plt.ylabel('Testing Loss')
plt.title('Effect of Width Shift Range on Testing Loss')

plt.tight_layout()
plt.show()


In [ ]:
# Data augmentation --- part 3
# This cell is to fine tune the height shift range on the testing accuercy and loss

# Define a range of zoom levels to test
height_shift_ranges = [0.1, 0.2, 0.3, 0.4, 0.5]

# Lists to store testing accuracies and losses for each zoom range
testing_accuracies = []
testing_losses = []

for height_range in height_shift_ranges:
    # Create an instance of ImageDataGenerator with the current zoom range
    datagen = ImageDataGenerator(
        zoom_range=0.5,
        fill_mode='nearest',
        width_shift_range=0.1,
        height_shift_range=height_range
    )

    # Apply data augmentation to the entire training dataset
    augmented_train_data = datagen.flow(train_images, train_labels, batch_size=32)

    # Combine original training data with augmented data
    combined_train_data = np.concatenate([train_images, augmented_train_data[0][0]], axis=0)
    combined_train_labels = np.concatenate([train_labels, augmented_train_data[0][1]], axis=0)

    # Create an instance of the Adam optimizer with the specified learning rate
    custom_optimizer = Adam(learning_rate=learning_rate)

    # Build the CNN model
    new_model = build_cnn_model()

    # Compile the model
    new_model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model using combined data
    new_model.fit(
        combined_train_data,
        combined_train_labels,
        batch_size=16,
        epochs=epochs,
        validation_data=(test_images, test_labels),
        verbose=0
    )

    # Evaluate the model on the test dataset
    test_loss, test_accuracy = new_model.evaluate(test_images, test_labels)
    testing_accuracies.append(test_accuracy)
    testing_losses.append(test_loss)

# Plot the testing accuracies against zoom ranges
plt.figure(figsize=(12, 4))

# Plot Testing Accuracy
plt.subplot(1, 2, 1)
plt.plot(height_shift_ranges, testing_accuracies, marker='o')
plt.xlabel('height Shift Range')
plt.ylabel('Testing Accuracy')
plt.title('Effect of height Shift Range on Testing Accuracy')

# Plot Testing Loss
plt.subplot(1, 2, 2)
plt.plot(height_shift_ranges, testing_losses, marker='o', color='orange')
plt.xlabel('height Shift Range')
plt.ylabel('Testing Loss')
plt.title('Effect of height Shift Range on Testing Loss')

plt.tight_layout()
plt.show()


In [ ]:
# Data augmentation --- part 4
# This cell is to fine tune the rotation range on the testing accuercy and loss

# Define a range of rotation angles to test
rotation_ranges = [10, 15, 20, 25, 30]

# Lists to store testing accuracies and losses for each rotation range
testing_accuracies = []
testing_losses = []

for rotation_range in rotation_ranges:
    # Create an instance of ImageDataGenerator with the current rotation range
    datagen = ImageDataGenerator(
        width_shift_range=0.1,
        height_shift_range=0.2,
        zoom_range=0.5,
        fill_mode='nearest',
        rotation_range=rotation_range
    )

    # Apply data augmentation to the entire training dataset
    augmented_train_data = datagen.flow(train_images, train_labels, batch_size=32)

    # Combine original training data with augmented data
    combined_train_data = np.concatenate([train_images, augmented_train_data[0][0]], axis=0)
    combined_train_labels = np.concatenate([train_labels, augmented_train_data[0][1]], axis=0)

    # Create an instance of the Adam optimizer with the specified learning rate
    custom_optimizer = Adam(learning_rate=learning_rate)

    # Build the CNN model
    new_model = build_cnn_model()

    # Compile the model
    new_model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model using combined data
    new_model.fit(
        combined_train_data,
        combined_train_labels,
        batch_size=16,
        epochs=epochs,
        validation_data=(test_images, test_labels),
        verbose=0
    )

    # Evaluate the model on the test dataset
    test_loss, test_accuracy = new_model.evaluate(test_images, test_labels)
    testing_accuracies.append(test_accuracy)
    testing_losses.append(test_loss)

# Plot the testing accuracies against rotation ranges
plt.figure(figsize=(12, 4))

# Plot Testing Accuracy
plt.subplot(1, 2, 1)
plt.plot(rotation_ranges, testing_accuracies, marker='o')
plt.xlabel('Rotation Range')
plt.ylabel('Testing Accuracy')
plt.title('Effect of Rotation Range on Testing Accuracy')

# Plot Testing Loss
plt.subplot(1, 2, 2)
plt.plot(rotation_ranges, testing_losses, marker='o', color='orange')
plt.xlabel('Rotation Range')
plt.ylabel('Testing Loss')
plt.title('Effect of Rotation Range on Testing Loss')

plt.tight_layout()
plt.show()


In [ ]:
# Data augmentation ----part 5
# This cell is to use the data augmentation with the model specified before
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Create an instance of ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range=0.2,
    zoom_range = 0.5,
    fill_mode = 'nearest'
)

# Apply data augmentation to the entire training dataset
augmented_train_data = datagen.flow(train_images, train_labels, batch_size=32)

# Combine original training data with augmented data
combined_train_data = np.concatenate([train_images, augmented_train_data[0][0]], axis=0)
combined_train_labels = np.concatenate([train_labels, augmented_train_data[0][1]], axis=0)

# Create an instance of the Adam optimizer with the specified learning rate
custom_optimizer = Adam(learning_rate=learning_rate)

# Build the CNN model
new_model = build_cnn_model()

# Compile the model
new_model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Now, train the model using combined data
augmented_history = new_model.fit(
    combined_train_data,  # Use combined training data
    combined_train_labels,  # Use combined training labels
    batch_size=16,
    epochs=epochs,
    validation_data=(test_images, test_labels),
)

test_loss, test_accuracy = new_model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Plot the training history (loss and accuracy)
plt.figure(figsize=(12, 4))

# Plot Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(augmented_history.history['loss'], label='Training Loss')
plt.plot(augmented_history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(augmented_history.history['accuracy'], label='Training Accuracy')
plt.plot(augmented_history.history['val_accuracy'], label='Testing Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Testing Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# This cell is to resize images so that they can be used in AlexNet model
import cv2

# Define the desired output size
output_size = (227, 227)

# Function to resize images
def resize_images(images):
    resized_images = []
    for image in images:
        # Convert single-channel image to 3 channels
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        # Resize the image to the desired output size
        resized_image = cv2.resize(image, output_size)
        # Append the resized image to the list
        resized_images.append(resized_image)
    return np.array(resized_images)

# Resize the images
resized_combined_train_data = resize_images(combined_train_data)
resized_test_images = resize_images(test_images)

# Convert data to TensorFlow tensors and move to GPU
resized_combined_train_data = tf.convert_to_tensor(resized_combined_train_data, dtype=tf.float32)
resized_test_images = tf.convert_to_tensor(resized_test_images, dtype=tf.float32)
combined_train_labels = tf.convert_to_tensor(combined_train_labels, dtype=tf.float32)
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.float32)

In [ ]:
# This cell is used to define and train the AlexNet model
# Define the AlexNet model
def create_alexnet(input_shape=(227, 227, 3), num_classes=28):
    model = Sequential()

    # Layer 1
    model.add(Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Layer 2
    model.add(Conv2D(256, (5, 5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Layer 3
    model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))

    # Layer 4
    model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))

    # Layer 5
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Flatten
    model.add(Flatten())

    # Layer 6
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    # Layer 7
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Create an instance of the Adam optimizer with the specified learning rate
custom_optimizer = Adam(learning_rate=learning_rate)

# Create the AlexNet model
alexnet_model = create_alexnet()

# Compile the model
alexnet_model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Now, train the model using combined data
alexNet_history = alexnet_model.fit(
    resized_combined_train_data,  # Use combined training data
    combined_train_labels,  # Use combined training labels
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(resized_test_images, test_labels),
)

test_loss, test_accuracy = alexnet_model.evaluate(resized_test_images, test_labels)
# print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Plot the training history (loss and accuracy)
plt.figure(figsize=(12, 4))

# Plot Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(alexNet_history.history['loss'], label='Training Loss')
plt.plot(alexNet_history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(alexNet_history.history['accuracy'], label='Training Accuracy')
plt.plot(alexNet_history.history['val_accuracy'], label='Testing Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Testing Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

# Convert grayscale images to RGB
combined_train_data_new = np.repeat(combined_train_data, 3, axis=-1)
test_images_new = np.repeat(test_images, 3, axis=-1)

# Choose a pre-trained model (VGG16 in this example)
weights_path = "/kaggle/input/vgg16-weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model = VGG16(weights=weights_path, include_top=False, input_shape=(32, 32, 3))

# Add a new classification layer for 28 classes
model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(28, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    combined_train_data_new,  # Use combined training data
    combined_train_labels,  # Use combined training labels
    #steps_per_epoch=len(combined_train_data) // 16,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(test_images_new, test_labels),
    #validation_steps=len(test_images) // 16
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images_new, test_labels)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Plot the training history (loss and accuracy)
plt.figure(figsize=(12, 4))

# Plot Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Testing Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Testing Accuracy')
plt.legend()

plt.tight_layout()
plt.show()
